# 0-2. 여성 월별 거주인구 -> 일별 거주인구

- 거주인구데이터가 월별로 되어있기 때문에 에측에 사용하기 위해 일별 변환
- 정규분포를 따르는 난수추출 방식을 사용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import time

In [2]:
Data_5 = pd.read_csv('../data/train_data/03_거주인구_RESIDENT_POP.CSV',encoding='cp949')

In [3]:
Data_5 = Data_5.astype({'base_year': 'str', 'base_month' : 'str'})
Data_5['base_date'] = Data_5['base_year'] + '-' + Data_5['base_month']

In [4]:
Data_5['base_date'] = Data_5['base_date'].apply(lambda x: x.replace('-','-0') if len(x)==6 else x)

In [5]:
Data_5 = Data_5.drop(['city','emd_cd','base_year','base_month'],axis=1)

In [6]:
Data_5 = Data_5.groupby(['base_date','emd_nm','sex']).sum().reset_index()

In [7]:
Data_5 = Data_5[Data_5['sex']=='여성']
Data_5.drop('sex',axis=1,inplace=True)

In [8]:
Data_5 = Data_5.drop(Data_5[Data_5.emd_nm == '우도면'].index)

In [9]:
Data_5 = Data_5.drop(Data_5[Data_5.emd_nm == '추자면'].index)

In [10]:
Date = pd.date_range('2018-01-01','2021-06-30',freq="D")

In [11]:
emd = Data_5.emd_nm.unique()

In [12]:
Date_df = pd.DataFrame(Date)
Date_df.columns = ['Date']

In [13]:
emd_df = pd.DataFrame(emd)
emd_df.columns = ['emd_nm']

In [14]:
Date_new = []
for i in range(len(Date)):
    Date_new.append(str(Date[i]))

In [15]:
Data_renew = []
for i in range(len(Date_new)):
    Data_renew.append(Date_new[i][:10])

In [16]:
import itertools
result = itertools.product(Data_renew, emd)
result = list(result)

In [17]:
len(result)

52357

In [18]:
len(emd)

41

In [19]:
len(Data_renew)

1277

In [20]:
df = pd.DataFrame(list(result))
df.columns = ['base_date','emd_nm']

In [21]:
df = df.sort_values(by=['emd_nm','base_date'])
df = df.reset_index()
df = df.drop('index',axis=1)

In [22]:
Data_5 = Data_5.sort_values(by=['emd_nm','base_date'])
Data_5 = Data_5.reset_index()
Data_5 = Data_5.drop('index',axis=1)

In [23]:
size = [31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30]

In [24]:
size = size*41

In [25]:
resid_pop = []
for i in range(len(Data_5)):
    a=list(np.random.normal(Data_5['resid_reg_pop'][i], 50,size=size[i]).astype('int'))
    resid_pop.extend(a)

In [26]:
df['resid_pop'] = resid_pop

In [27]:
foreign_pop = []
for i in range(len(Data_5)):
    a=list(np.random.normal(Data_5['foreign_pop'][i], size=size[i]).astype('int'))
    foreign_pop.extend(a)

In [28]:
df['foreign_pop'] = foreign_pop

In [29]:
df['total_pop'] = df['resid_pop'] + df['foreign_pop']

In [30]:
df

,base_date,emd_nm,resid_pop,foreign_pop,total_pop
0,2018-01-01,건입동,4659,81,4740
1,2018-01-02,건입동,4727,81,4808
2,2018-01-03,건입동,4738,82,4820
3,2018-01-04,건입동,4633,83,4716
4,2018-01-05,건입동,4736,81,4817
...,...,...,...,...,...
52352,2021-06-26,효돈동,2605,18,2623
52353,2021-06-27,효돈동,2534,19,2553
52354,2021-06-28,효돈동,2602,18,2620
52355,2021-06-29,효돈동,2613,19,2632


## 0-1-1. emd_nm 의 '알수없음'은 해당일 전체 읍면동 거주인구를 평균을 냄

In [31]:
unknown = df.groupby('base_date').mean().reset_index()

In [32]:
unknown['emd_nm'] = '알수없음'

In [33]:
unknown['resid_pop'] = unknown['resid_pop'].astype('int')

In [34]:
unknown['foreign_pop'] = unknown['foreign_pop'].astype('int')

In [35]:
unknown['total_pop'] = unknown['total_pop'].astype('int')

In [36]:
unknown

,base_date,resid_pop,foreign_pop,total_pop,emd_nm
0,2018-01-01,7934,237,8171,알수없음
1,2018-01-02,7939,237,8176,알수없음
2,2018-01-03,7937,237,8174,알수없음
3,2018-01-04,7935,237,8172,알수없음
4,2018-01-05,7929,237,8166,알수없음
...,...,...,...,...,...
1272,2021-06-26,8175,231,8406,알수없음
1273,2021-06-27,8168,231,8400,알수없음
1274,2021-06-28,8174,232,8406,알수없음
1275,2021-06-29,8178,231,8409,알수없음


In [37]:
df_final = pd.concat([df,unknown])

In [40]:
df_final

,base_date,emd_nm,resid_pop,foreign_pop,total_pop
0,2018-01-01,건입동,4659,81,4740
1,2018-01-02,건입동,4727,81,4808
2,2018-01-03,건입동,4738,82,4820
3,2018-01-04,건입동,4633,83,4716
4,2018-01-05,건입동,4736,81,4817
...,...,...,...,...,...
1272,2021-06-26,알수없음,8175,231,8406
1273,2021-06-27,알수없음,8168,231,8400
1274,2021-06-28,알수없음,8174,232,8406
1275,2021-06-29,알수없음,8178,231,8409


In [41]:
df_final.columns = ['일자', '읍면동', '여_내거주', '여_외거주', '여_총거주']

In [42]:
df_final.to_csv('../data/processed_data/여성_거주인구.csv',encoding='cp949',index=False)